# Function

In [285]:
import pandas as pd
import numpy as np
signal = pd.read_pickle("C:/Users/86773/Desktop/Signal_588000/20201204.txt")
#1/10000 cost
#PNL

In [250]:
def strategy_ret(ask_bid): # 用strategy得到的return
    total_cost = 0
    max_cost = 0
    position = 0
    time = 0
    num_trans = 0
    hold = '' #是否在开着单
    for abt in ask_bid:
        if abt == 0:
            continue;
        elif abt > 0:
            position += 1000;
            total_cost += 1.0001 * 1000 * abt # (1+换手率) * 1手 * ask
            num_trans += 1
        elif abt < 0:
            position -= 1000;
            total_cost += 0.9999 * 1000 * abt #(1-换手率) * 1手 * bid
            num_trans += 1
        max_cost = max(max_cost, total_cost)
    return [total_cost, position, max_cost, num_trans]

In [304]:
def rolling_quantile(lst):
    long = (lst >= lst.rolling(window=20).quantile(0.9))# & (lst > 0)
    short = (lst <= lst.rolling(window=20).quantile(0.1))# & (lst < 0)
    return long, short

# Individual

In [305]:
long, short = rolling_quantile(signal['PredictY'])
long = long.shift(1)
short = short.shift(1)
ask = signal['AskPrice1']
bid = signal['BidPrice1']

In [306]:
net_ask_bid = ask*long - bid*short

In [307]:
net_ask_bid

time
2020-12-04 09:35:02      NaN
2020-12-04 09:35:05        0
2020-12-04 09:35:08        0
2020-12-04 09:35:11        0
2020-12-04 09:35:14        0
                       ...  
2020-12-04 14:44:46    1.409
2020-12-04 14:44:49    1.409
2020-12-04 14:44:52        0
2020-12-04 14:44:55        0
2020-12-04 14:44:58        0
Length: 4499, dtype: object

In [308]:
[total_cost, position, max_cost, num_trans] = strategy_ret(net_ask_bid)

In [309]:
position * signal['BidPrice1'][signal.shape[0] - 1] * 0.9999 - total_cost

62594.07210000008

In [310]:
max_cost

93681.60779999997

# All

In [311]:
import os

directory = "C:/Users/86773/Desktop/Signal_588000"
total_gain = 0
num_win = 0
total_loss = 0
num_loss = 0
lst_max_cost = []
lst_net_value = []
lst_date = []
lst_num_trans = []
lst_sdv = []
for filename in os.listdir(directory):
    lst_date.append(filename[0:8])
    file = directory + '/' + filename
    signal_temp = pd.read_pickle(file)
    ask = signal_temp['AskPrice1']
    bid = signal_temp['BidPrice1']
    long, short = rolling_quantile(signal_temp['PredictY'])
    net_ask_bid = ask*long - bid*short
    [total_cost, position, max_cost, num_trans] = strategy_ret(net_ask_bid)
    net_value = position * signal_temp['BidPrice1'][signal_temp.shape[0] - 1] * 0.9999 - total_cost
    lst_net_value.append(net_value)
    lst_max_cost.append(max_cost)
    lst_num_trans.append(num_trans)
    lst_sdv.append(np.std(signal_temp['Ret']))
    if net_value > 0:
        num_win += 1
        total_gain += net_value
    elif net_value <= 0:
        num_loss += 1
        total_loss -= net_value

In [312]:
total_gain / num_win / (total_loss / num_loss)

70.85200848605106

In [313]:
result_588000 = pd.DataFrame({'date': lst_date, 'max cost': lst_max_cost, 'net value': lst_net_value, 'num trans': lst_num_trans, 'sdv': lst_sdv})
result_588000.to_csv('result_588000.csv')